In [1]:
## Import the packages
from collections import Counter
from sklearn.metrics import roc_auc_score, f1_score, fbeta_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import lightgbm as lgb
from functools import reduce
import os

## Import the source dataset

In [2]:
## Set the data folder path
folder_path = '../data/'

In [3]:
# Peek what is in the folder 
os.listdir(folder_path)

['feature_v5.csv',
 'feature_v4.csv',
 'feature_v3.csv',
 '.DS_Store',
 'total_vairables.csv',
 'submission.csv',
 'user_info.csv',
 'purchase_detail.csv',
 'login.csv',
 'month_amount.csv',
 'user_label_train.csv',
 'rfm_features.csv',
 'growth_features.csv',
 'source_data.zip']

In [4]:
# Import the source data
df_train = pd.read_csv(os.path.join(folder_path, 'user_label_train.csv'))
df_test = pd.read_csv(os.path.join(folder_path, 'submission.csv'))
df_user_info = pd.read_csv(os.path.join(folder_path, 'user_info.csv'))
df_login = pd.read_csv(os.path.join(folder_path, 'login.csv'))
df_purchase_detail = pd.read_csv(os.path.join(folder_path, 'purchase_detail.csv'))

In [5]:
# Import the auxilary dataset created from feature engineering
#df_month = pd.read_csv(os.path.join(folder_path, 'month_amount.csv')) #Replaced by df_monthly_buy_sum_pivot
df_feature_v3 = pd.read_csv(os.path.join(folder_path, 'feature_v3.csv'))
df_feature_v4 = pd.read_csv(os.path.join(folder_path, 'feature_v4.csv'))
df_feature_v5 = pd.read_csv(os.path.join(folder_path, 'feature_v5.csv'))
df_growth_features = pd.read_csv(os.path.join(folder_path, 'growth_features.csv'))
df_rfm_features = pd.read_csv(os.path.join(folder_path, 'rfm_features.csv'))

## Know Your Data: Exploratory Data Analysis

In [6]:
df_train['label'].value_counts(normalize=True)

0    0.658207
1    0.341793
Name: label, dtype: float64

In [7]:
print('the number of train: {}'.format(len(df_train)))
print('the number of test: {}'.format(len(df_test)))

the number of train: 426832
the number of test: 75325


In [8]:
# Peek user_label_train
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426832 entries, 0 to 426831
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   userid  426832 non-null  int64
 1   label   426832 non-null  int64
dtypes: int64(2)
memory usage: 6.5 MB


In [9]:
# Peek user_info.csv
df_user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502157 entries, 0 to 502156
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userid       502157 non-null  int64  
 1   gender       502122 non-null  float64
 2   is_seller    502157 non-null  int64  
 3   birth_year   277496 non-null  float64
 4   enroll_time  502157 non-null  object 
dtypes: float64(2), int64(2), object(1)
memory usage: 19.2+ MB


In [10]:
# Peek login.csv
df_login.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50705586 entries, 0 to 50705585
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   userid       int64 
 1   date         object
 2   login_times  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ GB


In [11]:
# Peek purchase_detail.csv
df_purchase_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7792956 entries, 0 to 7792955
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   userid            int64 
 1   grass_date        object
 2   order_count       int64 
 3   total_amount      int64 
 4   category_encoded  int64 
dtypes: int64(4), object(1)
memory usage: 297.3+ MB


In [12]:
# Peek submission.csv
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75325 entries, 0 to 75324
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   userid  75325 non-null  int64
dtypes: int64(1)
memory usage: 588.6 KB


## Feature Engineering

In [13]:
# Prepare for the train and test dataset with synchronous pre-process
df_train = pd.merge(df_train, df_user_info, on='userid', how='inner')
df_test = pd.merge(df_test, df_user_info, on='userid', how='inner')

print('The number of train: {}'.format(len(df_train)))
print('The number of test: {}'.format(len(df_test)))

The number of train: 426832
The number of test: 75325


In [14]:
# Process the age from the birth_year
df_train['age'] = 2020 - df_train.birth_year
df_test['age'] = 2020 - df_test.birth_year

# Process the enroll time to `pd.datetime` type
df_train['enroll_time'] = pd.to_datetime(df_train['enroll_time'])
df_test['enroll_time'] = pd.to_datetime(df_test['enroll_time'])

# Process the account lifetime to df_train and df_test respectively
df_train['lifetime'] = pd.to_datetime('2020-07-31') - pd.to_datetime(df_train['enroll_time'])
df_train.lifetime = df_train.lifetime.astype('timedelta64[D]')

df_test['lifetime'] = pd.to_datetime('2020-07-31') - pd.to_datetime(df_test['enroll_time'])
df_test.lifetime = df_test.lifetime.astype('timedelta64[D]')

### - Preliminarily Process the df_login (login.csv)

In [15]:
# Process the login log to count total login times, min/max login times for a day, std and mean for overall login times
df_login_feature = df_login.groupby('userid').agg({'login_times':['sum', 'min', 'max', 'std', 'mean']})
df_login_feature.columns = ["_".join(x) for x in df_login_feature.columns.ravel()] #automatically add column names

### - Preliminarily Process the df_purchase_detail (purchase_detail.csv)

In [16]:
# See the purchase detail of a certain user and know how data sorted
df_purchase_detail[df_purchase_detail.userid == 295790]

,userid,grass_date,order_count,total_amount,category_encoded
0,295790,2020-05-13,4,4,8
2928017,295790,2020-06-11,27,48,8
2931480,295790,2020-03-18,2,2,6
2934264,295790,2020-04-22,3,4,6
2938794,295790,2020-06-11,1,4,15
2939318,295790,2020-06-29,2,2,9
2940983,295790,2020-06-29,1,1,6
2955053,295790,2020-07-29,1,1,6
2959250,295790,2020-03-30,1,1,22
2960153,295790,2020-06-11,2,4,22


In [17]:
# Process the field `grass_date` to be datatime type 
df_purchase_detail['grass_date'] = pd.to_datetime(df_purchase_detail.grass_date)

# Sort the purchase detail by userid and purchase date 
df_purchase_detail = df_purchase_detail.sort_values(['userid', 'grass_date'], ascending=[1, 1])

In [18]:
# Process the purchase time to see the freqency of a user's purchasement
df_dt = df_purchase_detail[['userid','grass_date']].drop_duplicates()
df_dt['dt_diff'] = df_dt.groupby('userid')['grass_date'].diff().astype('timedelta64[D]')
df_purchase_dt_diff = df_dt.groupby('userid').agg({'dt_diff':['sum', 'min', 'max', 'std', 'mean']})
df_purchase_dt_diff.columns = ["_".join(x) for x in df_purchase_dt_diff.columns.ravel()]

df_purchase_dt_diff.head()

,dt_diff_sum,dt_diff_min,dt_diff_max,dt_diff_std,dt_diff_mean
userid,,,,,
1,174.0,3.0,50.0,17.705931,19.333333
2,147.0,1.0,28.0,8.473942,10.500000
3,94.0,3.0,41.0,14.854853,15.666667
4,170.0,1.0,16.0,4.220911,5.666667
5,171.0,1.0,18.0,3.603718,3.886364


In [19]:
# Calculate the monthly purchase summary for each user (Replace original df_month)
df_purchase_detail['month'] = df_purchase_detail.grass_date.dt.month

df_monthly_buy = df_purchase_detail.groupby(['userid', 'month']).agg({'order_count':['sum', 'mean'], 'total_amount':['sum', 'mean']})
df_monthly_buy.columns = ["_".join(x) for x in df_monthly_buy.columns.ravel()]

# Transform the monthly summary into pivot table to easily merge back to df_train and df_train
df_monthly_buy_sum_pivot = pd.pivot_table(df_monthly_buy, index='userid', columns='month', values='total_amount_sum') #.to_csv('month_amount.csv')
df_monthly_buy_sum_pivot.columns = [ f'month_{x}_sum_pivot' for x in df_monthly_buy_sum_pivot.columns.ravel()]

df_monthly_buy_avg_pivot = pd.pivot_table(df_monthly_buy, index='userid', columns='month', values='total_amount_mean') #.to_csv('month_total_amount_mean.csv')
df_monthly_buy_avg_pivot.columns = [ f'month_{x}_avg_pivot' for x in df_monthly_buy_avg_pivot.columns.ravel()]

df_monthly_buy_sum_pivot.head()

,month_2_sum_pivot,month_3_sum_pivot,month_4_sum_pivot,month_5_sum_pivot,month_6_sum_pivot,month_7_sum_pivot
userid,,,,,,
1,170.0,NaN,29.0,2.0,5.0,40.0
2,NaN,10.0,8.0,19.0,10.0,12.0
3,NaN,2.0,2.0,3.0,7.0,NaN
4,13.0,5.0,12.0,12.0,25.0,15.0
5,14.0,27.0,2597.0,8.0,20.0,68.0


In [20]:
# Calculate the total n purchase days of each user 
df_purchase_ndays = df_purchase_detail.groupby('userid')['grass_date'].count()

In [21]:
# Calculate the average purchase amount per order of each user
df_purchase_detail['amount_per_order'] = df_purchase_detail.total_amount / df_purchase_detail.order_count
df_purchase_amount_per_order = df_purchase_detail.groupby('userid').agg({'amount_per_order':['sum', 'min', 'max', 'std','mean']})
df_purchase_amount_per_order.columns = ["_".join(x) for x in df_purchase_amount_per_order.columns.ravel()]

# Calculate the statistics of the purchase of each user as new features
df_purchase_detail_feature = df_purchase_detail.groupby('userid').agg({'order_count':['sum', 'min', 'max', 'std', 'mean'],
                                                                       'total_amount':['sum', 'min', 'max', 'std','mean']})
df_purchase_detail_feature.columns = ["_".join(x) for x in df_purchase_detail_feature.columns.ravel()]

In [22]:
# Count the total purchase amount of each category for each user
df_category_count = df_purchase_detail.groupby(['userid','category_encoded'], as_index=False)['total_amount'].sum()
df_category_pivot = pd.pivot_table(df_category_count, index='userid', 
                                                      columns='category_encoded',
                                                      values='total_amount')
df_category_pivot = df_category_pivot.fillna(0)
df_category_pivot.columns = ['category_' + str(x) for x in df_category_pivot.columns]

### - Apply RFM Model to synethesize the features (WIP)
RFM stands for *Receency*, *Frequency*, and *Monetary*. But we don't have the price for each product category `category_encoded`, we then use total amount to calculate for the *Monetary* feature. Also, we utilized `login.csv` to calculate users' login behavior -- *Receency* and *Frequency*.

In [23]:
df_login.head()

,userid,date,login_times
0,161097,2020-07-29,2
1,243570,2020-07-29,2
2,355497,2020-07-29,4
3,167925,2020-07-29,1
4,504766,2020-07-29,1


In [24]:
# Login Recency
#df_login_recency = pd.DataFrame(df_login.groupby('userid').max()['date'])
#df_login_recency['login_recency'] = pd.to_datetime('2020-07-31')- pd.to_datetime(df_login_recency['date']) # the end date of this dataset is 2020/07/31

In [25]:
# (WIP) Other RFM features are produced in R code, which will be transformed to python code in the future 

In [26]:
# Select the desired features from the dataset created from R code
df_rfm_features = df_rfm_features[['userid', 'DistinctDay', 'DistinctDayIn90days', 'DistinctDayIn60days',
       'DistinctDayIn30days', 'DistinctDayIn14days', 'DistinctDayIn7days',
       'DistinctDayIn3days', 'FreqIn90days', 'FreqIn60days',
       'FreqIn14days', 'FreqIn3days', 'FreqIn7days', 'rececny']]

## Integrate all systhesized featurs into both the `df_train` and `df_test` datasets

In [27]:
# Merge synthesized features back into df_train and df_test
df_list = [df_login_feature, df_purchase_dt_diff, df_feature_v3, df_feature_v4, df_feature_v5, df_monthly_buy_sum_pivot, df_purchase_ndays, 
           df_purchase_detail_feature, df_purchase_amount_per_order, df_growth_features, df_rfm_features, df_category_pivot] #remove `df_month` and add `df_monthly_buy_sum_pivot`

# Applied `reduce` function to merge the orginal df_train/df_test and created features
df_train = reduce(lambda left, right: pd.merge(left, right, on='userid', how='inner'), [df_train, *df_list])
df_test = reduce(lambda left, right: pd.merge(left, right, on='userid', how='inner'), [df_test, *df_list])

## Modeling with Lightgbm

In [28]:
# List out the current sythesized features on train and test dataset
df_train.columns 

Index(['userid', 'label', 'gender', 'is_seller', 'birth_year', 'enroll_time',
       'age', 'lifetime', 'login_times_sum', 'login_times_min',
       'login_times_max', 'login_times_std', 'login_times_mean', 'dt_diff_sum',
       'dt_diff_min', 'dt_diff_max', 'dt_diff_std', 'dt_diff_mean',
       'AvgMoMOrderCnt', 'AvgMoMTotCnt', 'OrderCntIn90days', 'TotCntIn90days',
       'OrderCntIn60days', 'TotCntIn60days', 'OrderCntIn30days',
       'TotCntIn30days', 'OrderCntIn14days', 'TotCntIn14days',
       'OrderCntIn7days', 'TotCntIn7days', 'OrderCntIn3days', 'TotCntIn3days',
       'BuyRececny', 'DistinctCategory', 'GoodBuyer', 'month_2_sum_pivot',
       'month_3_sum_pivot', 'month_4_sum_pivot', 'month_5_sum_pivot',
       'month_6_sum_pivot', 'month_7_sum_pivot', 'grass_date',
       'order_count_sum', 'order_count_min', 'order_count_max',
       'order_count_std', 'order_count_mean', 'total_amount_sum',
       'total_amount_min', 'total_amount_max', 'total_amount_std',
       'total_amoun

In [29]:
# Split the train and test datasets
df_tr, df_val = train_test_split(df_train, stratify = df_train['label'], test_size=0.2, random_state=42)

In [30]:
# List out the features for traing
features = ['gender', 'is_seller', 'birth_year', 'age', 'lifetime', 
            'login_times_sum', 'login_times_min', 'login_times_max', 
            'login_times_std', 'login_times_mean', 'dt_diff_sum',
            'dt_diff_min', 'dt_diff_max', 'dt_diff_std', 'dt_diff_mean',
            'AvgMoMOrderCnt', 'AvgMoMTotCnt', 'OrderCntIn90days', 'TotCntIn90days',
            'OrderCntIn60days', 'TotCntIn60days', 'OrderCntIn30days',
            'TotCntIn30days', 'OrderCntIn14days', 'TotCntIn14days',
            'OrderCntIn7days', 'TotCntIn7days', 'OrderCntIn3days', 'TotCntIn3days',
            'BuyRececny', 'DistinctCategory', 'GoodBuyer', 'month_2_sum_pivot',
            'month_3_sum_pivot', 'month_4_sum_pivot', 'month_5_sum_pivot',
            'month_6_sum_pivot', 'month_7_sum_pivot', 'grass_date',
            'order_count_sum', 'order_count_min', 'order_count_max',
            'order_count_std', 'order_count_mean', 'total_amount_sum',
            'total_amount_min', 'total_amount_max', 'total_amount_std',
            'total_amount_mean', 'amount_per_order_sum', 'amount_per_order_min',
            'amount_per_order_max', 'amount_per_order_std', 'amount_per_order_mean',
            'avgFreqMoM', 'DistinctDay', 'DistinctDayIn90days',
            'DistinctDayIn60days', 'DistinctDayIn30days', 'DistinctDayIn14days',
            'DistinctDayIn7days', 'DistinctDayIn3days', 'FreqIn90days',
            'FreqIn60days', 'FreqIn14days', 'FreqIn3days', 'FreqIn7days', 'rececny',
            'category_1', 'category_2', 'category_3', 'category_4', 'category_5',
            'category_6', 'category_7', 'category_8', 'category_9', 'category_10',
            'category_11', 'category_12', 'category_13', 'category_14',
            'category_15', 'category_16', 'category_17', 'category_18',
            'category_19', 'category_20', 'category_21', 'category_22',
            'category_23']

print(len(features))

91


In [31]:
# Create the training and validation sets
tr_X = df_tr[features].values
tr_y = df_tr['label']

val_X = df_val[features].values
val_y = df_val['label']

te_X = df_test[features].values

In [32]:
# Load the datasets into lgd.Dataset format
lgtrain = lgb.Dataset(tr_X, tr_y)
lgvalid = lgb.Dataset(val_X, val_y)

In [33]:
# Set the training parameters
params = {
          "objective" : "binary",
          "num_leaves" : 30,
          "max_depth": -1,
          "bagging_fraction" : 0.8,  # subsample
          "feature_fraction" : 0.8,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "num_threads":4,
          'lambda_l1': 0.9, 
          'lambda_l2': 0.5, 
          'learning_rate': 0.01, 
          'metric': 'AUC',
          'is_unbalance': False,
          "verbosity" : -1 }

In [35]:
# Start training
evals_result = {}

clf = lgb.train(params,
                lgtrain,
                1500,
                valid_sets = [lgvalid, lgtrain],
                valid_names = ['validation', 'train'],
#               feval=lgb_fbeta_score,
                evals_result = evals_result,
                early_stopping_rounds = 200,
                verbose_eval = 50)

Training until validation scores don't improve for 200 rounds
[50]	train's auc: 0.685575	validation's auc: 0.681029
[100]	train's auc: 0.688563	validation's auc: 0.683216
[150]	train's auc: 0.691162	validation's auc: 0.685512
[200]	train's auc: 0.693754	validation's auc: 0.687768
[250]	train's auc: 0.696127	validation's auc: 0.68979
[300]	train's auc: 0.698264	validation's auc: 0.691498
[350]	train's auc: 0.700111	validation's auc: 0.692901
[400]	train's auc: 0.701737	validation's auc: 0.694049
[450]	train's auc: 0.703116	validation's auc: 0.694895
[500]	train's auc: 0.704414	validation's auc: 0.695697
[550]	train's auc: 0.705569	validation's auc: 0.696299
[600]	train's auc: 0.706619	validation's auc: 0.696796
[650]	train's auc: 0.707621	validation's auc: 0.697275
[700]	train's auc: 0.708579	validation's auc: 0.697684
[750]	train's auc: 0.70947	validation's auc: 0.698026
[800]	train's auc: 0.710307	validation's auc: 0.698264
[850]	train's auc: 0.711184	validation's auc: 0.698506
[900]	

## Predict the unknown label by the trained model 

In [36]:
# Pedict the labels
pred_test = clf.predict(te_X)

In [37]:
# Output the predict labels to `submission.csv`
sub = pd.read_csv(os.path.join(folder_path, 'submission.csv'))
sub['label'] = pred_test

sub.to_csv('submission_final.csv', index=False)